In [2]:
import pandas as pd
import numpy as np
import OpenEXR
import Imath

In [3]:
def csv_to_directivity_textures(csv_path, output_dir):
    """
    Convert Genelec CSV to one EXR texture per frequency band.
    
    Args:
        csv_path: Path to Genelec8020_normalized_magnitudes.csv
        output_dir: Directory to save texture files
    """
    # Read CSV
    df = pd.read_csv(csv_path)
    
    # Extract frequencies from header (skip first column)
    frequencies = df.columns[1:].astype(float)
    
    # Initialize 2D arrays for each frequency (theta=180, phi=360)
    # Note: CSV has varying phi samples per theta, so we'll need to interpolate
    textures = {freq: np.zeros((181, 360)) for freq in frequencies}
    
    # Parse position strings and populate arrays
    for idx, row in df.iterrows():
        pos = row.iloc[0]  # e.g., "P000T000"
        
        # Parse phi and theta
        phi = int(pos[1:4])   # P000 -> 0
        theta = int(pos[5:8]) # T000 -> 0
        
        # Fill in values for each frequency
        for freq_idx, freq in enumerate(frequencies):
            magnitude = row.iloc[freq_idx + 1]
            textures[freq][theta, phi] = magnitude
    
    # Handle missing data via interpolation
    for freq in frequencies:
        texture = textures[freq]
        
        # Check for zeros (missing data) and interpolate
        mask = texture == 0
        if np.any(mask):
            # Simple nearest-neighbor or bilinear interpolation
            from scipy.ndimage import binary_dilation
            from scipy.interpolate import griddata
            
            # Get valid points
            valid_coords = np.argwhere(~mask)
            valid_values = texture[~mask]
            
            # Get invalid points
            invalid_coords = np.argwhere(mask)
            
            # Interpolate
            if len(invalid_coords) > 0 and len(valid_coords) > 0:
                interpolated = griddata(
                    valid_coords, 
                    valid_values, 
                    invalid_coords, 
                    method='nearest'
                )
                texture[mask] = interpolated
        
        # Save as EXR (high precision, HDR)
        save_exr_texture(
            texture, 
            f"{output_dir}/genelec8020_directivity_{int(freq)}Hz.exr"
        )
        
        print(f"Created texture for {freq} Hz")

def save_exr_texture(data, filename):
    """
    Save 2D numpy array as single-channel EXR file.
    
    Args:
        data: numpy array of shape (height, width)
        filename: output path
    """
    height, width = data.shape
    
    # Create EXR header
    header = OpenEXR.Header(width, height)
    header['channels'] = {'Y': Imath.Channel(Imath.PixelType(OpenEXR.FLOAT))}
    
    
    # Convert to float32 and flatten
    data_flat = data.astype(np.float32).tobytes()
    
    # Write file
    exr = OpenEXR.OutputFile(filename, header)
    exr.writePixels({'Y': data_flat})
    exr.close()

In [4]:
# Usage:
csv_to_directivity_textures(
    "Genelec8020_normalized_magnitudes.csv",
    "../directivity_textures"
)

Created texture for 20.0 Hz
Created texture for 31.5 Hz
Created texture for 40.0 Hz
Created texture for 50.0 Hz
Created texture for 63.0 Hz
Created texture for 80.0 Hz
Created texture for 100.0 Hz
Created texture for 125.0 Hz
Created texture for 160.0 Hz
Created texture for 200.0 Hz
Created texture for 250.0 Hz
Created texture for 315.0 Hz
Created texture for 400.0 Hz
Created texture for 500.0 Hz
Created texture for 630.0 Hz
Created texture for 800.0 Hz
Created texture for 1000.0 Hz
Created texture for 1250.0 Hz
Created texture for 1600.0 Hz
Created texture for 2000.0 Hz
Created texture for 2500.0 Hz
Created texture for 3150.0 Hz
Created texture for 4000.0 Hz
Created texture for 5000.0 Hz
Created texture for 6300.0 Hz
Created texture for 8000.0 Hz
Created texture for 10000.0 Hz
Created texture for 12500.0 Hz
Created texture for 16000.0 Hz
Created texture for 20000.0 Hz
